## 3-Box Model of Ocean Carbon

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [2]:
# model parameters

# mixing
vmix_hilat = 6.0e14  # m3 yr-1
vmix_thermohaline = 6.3072e14  # m3 yr-1

# productivity
tau_hilat = 50
tau_lolat = 1

percent_CaCO3 = 30

# ocean averages
total_PO4 = 2.5
total_DIC = 2225
total_TA = 2300

total_Ca = 10.2e-3  # mol kg

In [3]:
# set up boxes
lolat_surf = {'temp': 25.,
              'sal': 35.,
              'K0': .0285,
              'K1': 1.45e-6,
              'K2': 1.102e-9,
              'KW': 5.982e-14,
              'KB': 2.619e-9,
              'KspC': 4.2658e-7,
              'KspA': 6.45654e-7,
              'vol': 1.31e16}

hilat_surf = {'temp': 5.,
              'sal': 35.,
              'K0': 0.05241,
              'K1': 9.032e-7,
              'K2': 5.076e-10,
              'KW': 8.412e-15,
              'KB': 1.482e-9,
              'KspC': 4.2658e-7,
              'KspA': 6.45654e-7,
              'vol': 2.97e16}

deep = {'temp': 2.0,
        'sal': 35.,
        'K0': 0.05855,
        'K1': 8.279e-7,
        'K2': 4.475e-10,
        'KW': 6.068e-15,
        'KB': 1.349e-9,
        'KspC': 7.76e-7,
        'KspA': 1.2e-6,
        'vol': 1.25e18}

In [4]:
# carbon system approximations
def calc_CO3(DIC, TA):
    return TA - DIC

def calc_HCO3(DIC, TA):
    return 2 * DIC - TA

def calc_pCO2(CO3, HCO3, K0, K1, K2):
    return K2 * HCO3**2 / (K1 * K0 * CO3)

def calc_H(CO3, HCO3, K2):
    return K2 * HCO3 / CO3

## PO<sub>4</sub> Model Equations

$$
\begin{align}
\frac{d[PO_4]_{deep}}{dt} &= \frac{v_{mix} [PO_4]_{hilat} - v_{mix} [PO_4]_{deep} + v_{thermo} [PO_4]_{hilat} - v_{thermo} [PO_4]_{deep} + \frac{[PO_4]_{hilat} vol_{hilat}}{\tau_{hilat}} + \frac{[PO_4]_{lolat} vol_{lolat}}{\tau_{lolat}}}{vol_{deep}}\\
\frac{d[PO_4]_{hilat}}{dt} &= \frac{v_{mix} [PO_4]_{deep} - v_{mix} [PO_4]_{hilat} + v_{thermo} [PO_4]_{lolat} - v_{thermo} [PO_4]_{hilat} - \frac{[PO_4]_{hilat} vol_{hilat}}{\tau_{hilat}}}{vol_{hilat}} \\
\frac{d[PO_4]_{lolat}}{dt} &= \frac{v_{thermo} [PO_4]_{deep} - v_{thermo} [PO_4]_{lolat} - \frac{[PO_4]_{lolat} vol_{lolat}}{\tau_{lolat}}}{vol_{lolat}} \\
\end{align}
$$


In [5]:
def dPO4_deep(PO4_deep, PO4_hilat, PO4_lolat, vmix, vthermo, tau_hilat, vol_hilat, tau_lolat, vol_lolat, vol_deep):
    return (vmix * PO4_hilat - 
            vmix * PO4_deep + 
            vthermo * PO4_hilat - 
            vthermo * PO4_deep + 
            (PO4_hilat * vol_hilat / tau_hilat) + 
            (PO4_lolat * vol_lolat / tau_lolat)) / vol_deep

def dPO4_hilat(PO4_deep, PO4_hilat, PO4_lolat, vmix, vthermo, tau_hilat, vol_hilat):
    return (vmix * PO4_deep -
            vmix * PO4_hilat +
            vthermo * PO4_lolat -
            vthermo * PO4_hilat -
            (PO4_hilat * vol_hilat / tau_hilat)) / vol_hilat

def dPO4_lolat(PO4_deep, PO4_lolat, vthermo, tau_lolat, vol_lolat):
    return (vthermo * PO4_deep -
            vthermo * PO4_lolat -
            (PO4_lolat * vol_lolat / tau_lolat)) / vol_lolat

In [6]:
def zero_fn(X, total_PO4, vmix, vthermo, tau_hilat, tau_lolat, vol_hilat, vol_lolat, vol_deep):
    PO4_deep, PO4_hilat, PO4_lolat = X
    
    # calculate deltas to minimise
    d_deep = dPO4_deep(PO4_deep, PO4_hilat, PO4_lolat, vmix, vthermo, 
                       tau_hilat, vol_hilat, tau_lolat, vol_lolat, vol_deep)
    d_hilat = dPO4_hilat(PO4_deep, PO4_hilat, PO4_lolat, vmix, vthermo, tau_hilat, vol_hilat)
    d_lolat = dPO4_lolat(PO4_deep, PO4_lolat, vthermo, tau_lolat, vol_lolat)
    
    # calculate total PO4
    calc_total = ((PO4_deep * vol_deep + PO4_hilat * vol_hilat + PO4_lolat * vol_lolat) /
                  (vol_deep + vol_hilat + vol_lolat))
    # make sure it's the same as actual
    PO4_constraint = (total_PO4 - calc_total)**2
    
    # return value to minimise
    return (d_deep**2 + d_hilat**2 + d_lolat**2 + PO4_constraint)

In [7]:
guess = (1, 1, 1)
args = (total_PO4, vmix_hilat, vmix_thermohaline, tau_hilat, tau_lolat, hilat_surf['vol'], lolat_surf['vol'], deep['vol'])

fit = minimize(zero_fn, guess, args=args)

fit.x

array([2.56337211, 0.88358062, 0.11774838])

In [8]:
deep['PO4'], hilat_surf['PO4'], lolat_surf['PO4'] = fit.x

## Expand to Carbon

In [9]:
def calc_DIC_surf(PO4_deep, PO4_surf, percent_CaCO3, DIC_total):
    delta_PO4 = PO4_deep - PO4_surf
    
    organic = 106 * delta_PO4
    CaCO3 = 106 * percent_CaCO3 * delta_PO4 / 100
    
    return DIC_total - organic - CaCO3

def calc_TA_surf(PO4_deep, PO4_surf, percent_CaCO3, TA_total):
    delta_PO4 = PO4_deep - PO4_surf
    
    organic = 0.15 * 106 * delta_PO4
    CaCO3 = 2 * 106 * percent_CaCO3 * delta_PO4 / 100
    
    return TA_total - organic - CaCO3

In [10]:
hilat_surf['DIC'] = calc_DIC_surf(deep['PO4'], hilat_surf['PO4'], percent_CaCO3, total_DIC)
hilat_surf['TA'] = calc_TA_surf(deep['PO4'], hilat_surf['PO4'], percent_CaCO3, total_TA)

lolat_surf['DIC'] = calc_DIC_surf(deep['PO4'], lolat_surf['PO4'], percent_CaCO3, total_DIC)
lolat_surf['TA'] = calc_TA_surf(deep['PO4'], lolat_surf['PO4'], percent_CaCO3, total_TA)

deep['DIC'] = total_DIC #+ ((total_DIC - hilat_surf['DIC']) * hilat_surf['vol'] + (total_DIC - lolat_surf['DIC']) * lolat_surf['vol']) / deep['vol']
deep['TA'] = total_TA #+ ((total_TA - hilat_surf['TA']) * hilat_surf['vol'] + (total_TA - lolat_surf['TA']) * lolat_surf['vol']) / deep['vol']

In [11]:
for box in [hilat_surf, lolat_surf, deep]:
    box['CO3'] = calc_CO3(box['DIC'], box['TA'])
    box['HCO3'] = calc_HCO3(box['DIC'], box['TA'])
    box['pCO2'] = calc_pCO2(box['CO3'], box['HCO3'], box['K0'], box['K1'], box['K2'])
    box['H'] = calc_H(box['CO3'], box['HCO3'], box['K2'])
    box['pH'] = -np.log10(box['H'])
    box['Omega_C'] = box['CO3'] * 1e-6 * total_Ca / box['KspC']
    box['Omega_A'] = box['CO3'] * 1e-6 * total_Ca / box['KspA']

In [12]:
lolat_surf = pd.DataFrame(lolat_surf, index=[('Surface', 'lo lat')]).T

In [13]:
hilat_surf = pd.DataFrame(hilat_surf, index=[('Surface', 'hi lat')]).T

In [14]:
deep = pd.DataFrame(deep, index=[('Deep', '')]).T

In [15]:
model = pd.concat([hilat_surf, lolat_surf, deep], 1)

In [16]:
def boxmodel(vmix_hilat=6.0e14, vmix_thermohaline=6.3072e14, tau_hilat=50, tau_lolat=1, percent_CaCO3=30,
             total_PO4=2.5, total_DIC=2225, total_TA=2300, total_Ca=10.2e-3):
    # set up boxes
    lolat_surf = {'temp': 25.,
                  'sal': 35.,
                  'K0': .0285,
                  'K1': 1.45e-6,
                  'K2': 1.102e-9,
                  'KW': 5.982e-14,
                  'KB': 2.619e-9,
                  'KspC': 4.2658e-7,
                  'KspA': 6.45654e-7,
                  'vol': 1.31e16}

    hilat_surf = {'temp': 5.,
                  'sal': 35.,
                  'K0': 0.05241,
                  'K1': 9.032e-7,
                  'K2': 5.076e-10,
                  'KW': 8.412e-15,
                  'KB': 1.482e-9,
                  'KspC': 4.2658e-7,
                  'KspA': 6.45654e-7,
                  'vol': 2.97e16}

    deep = {'temp': 2.0,
            'sal': 35.,
            'K0': 0.05855,
            'K1': 8.279e-7,
            'K2': 4.475e-10,
            'KW': 6.068e-15,
            'KB': 1.349e-9,
            'KspC': 7.76e-7,
            'KspA': 1.2e-6,
            'vol': 1.25e18}
    
    guess = (1, 1, 1)
    args = (total_PO4, vmix_hilat, vmix_thermohaline, tau_hilat, tau_lolat, hilat_surf['vol'], lolat_surf['vol'], deep['vol'])

    fit = minimize(zero_fn, guess, args=args)

    deep['PO4'], hilat_surf['PO4'], lolat_surf['PO4'] = fit.x
    
    hilat_surf['DIC'] = calc_DIC_surf(deep['PO4'], hilat_surf['PO4'], percent_CaCO3, total_DIC)
    hilat_surf['TA'] = calc_TA_surf(deep['PO4'], hilat_surf['PO4'], percent_CaCO3, total_TA)

    lolat_surf['DIC'] = calc_DIC_surf(deep['PO4'], lolat_surf['PO4'], percent_CaCO3, total_DIC)
    lolat_surf['TA'] = calc_TA_surf(deep['PO4'], lolat_surf['PO4'], percent_CaCO3, total_TA)

    deep['DIC'] = total_DIC #+ ((total_DIC - hilat_surf['DIC']) * hilat_surf['vol'] + (total_DIC - lolat_surf['DIC']) * lolat_surf['vol']) / deep['vol']
    deep['TA'] = total_TA #+ ((total_TA - hilat_surf['TA']) * hilat_surf['vol'] + (total_TA - lolat_surf['TA']) * lolat_surf['vol']) / deep['vol']
    
    for box in [hilat_surf, lolat_surf, deep]:
        box['CO3'] = calc_CO3(box['DIC'], box['TA'])
        box['HCO3'] = calc_HCO3(box['DIC'], box['TA'])
        box['pCO2'] = calc_pCO2(box['CO3'], box['HCO3'], box['K0'], box['K1'], box['K2'])
        box['H'] = calc_H(box['CO3'], box['HCO3'], box['K2'])
        box['pH'] = -np.log10(box['H'])
        box['Omega_C'] = box['CO3'] * 1e-6 * total_Ca / box['KspC']
        box['Omega_A'] = box['CO3'] * 1e-6 * total_Ca / box['KspA']
    
    lolat_surf = pd.DataFrame(lolat_surf, index=[('Surface', 'lo lat')]).T
    hilat_surf = pd.DataFrame(hilat_surf, index=[('Surface', 'hi lat')]).T
    deep = pd.DataFrame(deep, index=[('Deep', '')]).T
    
    return pd.concat([lolat_surf, hilat_surf, deep], 1)

In [17]:
model = boxmodel(percent_CaCO3=30, tau_lolat=20)

print('Atmos: {:.0f}'.format(model.loc['pCO2', [('Surface', 'lo lat'), ('Surface', 'hi lat')]].mean()))

model.loc[['pCO2', 'pH', 'Omega_A', 'Omega_C'], :].T


Atmos: 448


,pCO2,pH,Omega_A,Omega_C
"(Surface, lo lat)",636.913585,7.857547,2.377765,3.598887
"(Surface, hi lat)",259.168051,8.189954,2.359438,3.571148
"(Deep, )",568.989057,7.891830,0.637500,0.985825
